## Working with filter pipelines

This Jupyter notebook explains the workflow of setting up and configuring a ground point filtering pipeline. This is an advanced workflow for users that want to define their own filtering workflows. For basic use, preconfigured pipelines are (or rather: will be) provided by `adaptivefiltering`. As always, we first need to import our library:

In [ ]:
import adaptivefiltering

Also, we need to load at least one data set which we will use to interactively preview our filter settings. Note that for a good interactive experience with no downtimes, you should restrict your datasets to a reasonable size (see the [Working with datasets](datasets.ipynb) notebook for how to do it).

In [ ]:
# dataset = adaptivefiltering.DataSet(filename="data/500k_NZ20_Westport.laz")
dataset = adaptivefiltering.DataSet(filename="data/uls_thingstaette.las")

### Filtering backends

`adaptivefiltering` does not implement its own ground point filtering algorithms. Instead, algorithms from existing packages are accessible through a common interface. Currently, the following backends are available:

* [PDAL](https://pdal.io/): The Point Data Abstraction Library is an open source library for point cloud processing.
* [OPALS](https://opals.geo.tuwien.ac.at/html/stable/index.html) is a proprietary library for processing Lidar data. It can be tested freely for datasets <1M points.

PDAL is always available when using `adaptivefiltering` and is used internally for many tasks that are not directly related to ground point filtering. In order to enable the OPALS backend, `adaptivefiltering` needs to be given the information where your OPALS installation (potentially including your license key) is located. This can either be done by setting the environment variable `OPALS_DIR` or by setting the path at runtime:

In [ ]:
adaptivefiltering.set_opals_directory("/path/to/opals")

### Configuring a filter pipeline

The main pipeline configuration is done by calling the `pipeline_tuning` function with your dataset as the parameter. This will open the interactive user interface which waits for your user input until you hit the *Finalize* button. The configured filter is then accessible as the Python object `pipeline`:

In [ ]:
pipeline = adaptivefiltering.pipeline_tuning(dataset)

If you want to inspect multiple data sets in parallel while tuning a pipeline, you can do so by passing a list of datasets to the `pipeline_tuning` function. Note that `adaptivefiltering` does currently not parallelize the execution of filter pipeline execution which may have a negative impact on wait times while tuning with multiple parameters.

In [ ]:
pipeline2 = adaptivefiltering.pipeline_tuning(datasets=[dataset, dataset])

### Storing and reloading filter pipelines

Pipeline objects can be stored on disk with the `save_filter` command from `adaptivefiltering`. We typically use the extension `json` for filters. It stands for *JavaScript Object Notation* and is a widely used format for storing custom data structures:

In [ ]:
adaptivefiltering.save_filter(pipeline, "myfilter.json")

The appropriate counterpart is `load_filter`, which restores the pipeline object from a file:

In [ ]:
old_pipeline = adaptivefiltering.load_filter("myfilter.json")

A filter pipeline loaded from a file can be edited using the `pipeline_tuning` command by passing it to the function. As always, the pipeline object returned by `pipeline_tuning` will be a new object - no implicit changes of the loaded pipeline object will occur:

In [ ]:
edited_pipeline = adaptivefiltering.pipeline_tuning(dataset, pipeline=old_pipeline)

### Applying filter pipelines to data

Pipeline objects can also be used to transform data sets by applying the ground point filtering algorithms. This is one of the core tasks of the `adaptivefiltering` library, but this will rarely be done in this manual fashion, as we will provide additional interfaces for (locally adaptive) application of filter pipelines:

In [ ]:
filtered = pipeline.execute(dataset)